# <center> Predicting customer churn for a bank using ANN <center>

### Introduction: 
In this project we have a bank that is losing customers and they want us to build a deep learning classfication model that can predict which customers are likely to leave next. The dataset is collected from 10,000 customers.

In [1]:
# importing librareis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline

In [2]:
# importing dataset
dataset = pd.read_csv("..Datasets\\Churn_Modelling.csv")
print(dataset.shape)
dataset.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Data preprocessing

The dataset contains 10000 samples and 14 independent variables. We will leave out variables that have no predictive value and that's the first three columns(RowNumber, CustomerId, Surname). We will also our matrix of features.

In [3]:
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, -1]
print(y.head(7))
X.head()

0    1
1    0
2    1
3    0
4    0
5    1
6    0
Name: Exited, dtype: int64


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [4]:
# checking features data types
X.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
dtype: object

We have two categorical features "Geography and Gender" that need to be encoded.

In [5]:
# importing library
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# encoding the Geography column
labelencoder_X_1 = LabelEncoder()
X.iloc[:, 1] = labelencoder_X_1.fit_transform(X.iloc[:, 1])

# encoding the Gender column
labelencoder_X_2 = LabelEncoder()
X.iloc[:, 2] = labelencoder_X_2.fit_transform(X.iloc[:, 2])

X.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10
5,645,2,1,44,8,113755.78,2,1,0,149756.71
6,822,0,1,50,7,0.00,2,1,1,10062.80
7,376,1,0,29,4,115046.74,4,1,0,119346.88
8,501,0,1,44,4,142051.07,2,0,1,74940.50
9,684,0,1,27,2,134603.88,1,1,1,71725.73


In [6]:
# creating a dummy variable for the multi class categorical feature "Geography"
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
# creating instance of one-hot-encoder
transformer = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), [1])],remainder='passthrough')
X = np.array(transformer.fit_transform(X), dtype=np.str)

X[:, 1:]

array([['0.0', '0.0', '619.0', ..., '1.0', '1.0', '101348.88'],
       ['0.0', '1.0', '608.0', ..., '0.0', '1.0', '112542.58'],
       ['0.0', '0.0', '502.0', ..., '1.0', '0.0', '113931.57'],
       ...,
       ['0.0', '0.0', '709.0', ..., '0.0', '1.0', '42085.58'],
       ['1.0', '0.0', '772.0', ..., '1.0', '0.0', '92888.52'],
       ['0.0', '0.0', '792.0', ..., '1.0', '0.0', '38190.78']],
      dtype='<U32')

In [7]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
X_train[10]

array([ 0.98560362, -0.5698444 , -0.57369368,  0.32421559, -1.09168714,
       -0.56031215,  1.04473698, -0.01778603, -0.92159124,  0.64259497,
        0.9687384 , -1.17149426])

In [10]:
X_test[10]

array([ 0.98560362, -0.5698444 , -0.57369368, -1.57262601, -1.09168714,
       -0.75172054,  1.73678764, -1.21571749,  0.8095029 ,  0.64259497,
       -1.03227043, -1.49960838])

In [11]:
# converting my data into arrays
X_train = np.asarray(X_train) 
X_test = np.asarray(X_test)
y_train = np.asarray(y_train) 
y_test = np.asarray(y_test)

### Building the ANN model

In [12]:
# importing deep learning libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Train on 8000 samples
Epoch 1/100
8000/8000 [==============================] - 9s 1ms/sample - loss: 0.4977 - accuracy: 0.7958
Epoch 2/100
8000/8000 [==============================] - 2s 247us/sample - loss: 0.4296 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 253us/sample - loss: 0.4236 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 250us/sample - loss: 0.4190 - accuracy: 0.8136
Epoch 5/100
8000/8000 [==============================] - 2s 254us/sample - loss: 0.4158 - accuracy: 0.8263
Epoch 6/100
8000/8000 [==============================] - 2s 273us/sample - loss: 0.4137 - accuracy: 0.8314
Epoch 7/100
8000/8000 [==============================] - 2s 254us/sample - loss: 0.4125 - accuracy: 0.8313
Epoch 8/100
8000/8000 [==============================] - 2s 249us/sample - loss: 0.4109 - accuracy: 0.8315
Epoch 9/100
8000/8000 [==============================] - 2s 254us/sample - loss: 0.4098 - accuracy: 0.8332
Epoch 10/100
8000

### Making predictions and evaluating the model

In [13]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Changing probabilities into true and false using 50% threshold
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [14]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1538,   57],
       [ 253,  152]], dtype=int64)

### Making predictions on a brand new customer

In [16]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)
new_prediction

array([[False]])

### Evaluating, Improving and Tuning the ANN

In [18]:
# Evaluating the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print(mean)
print(variance)

0.8398750066757202
0.010100280668603927


### Improving the ANN using Dropout Regularization to reduce overfitting if needed

In [20]:
# Tuning the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [20, 100],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
#rint(best_parameters)
#rint(best_accuracy)

Train on 7200 samples
Epoch 1/20
7200/7200 [==============================] - 2s 321us/sample - loss: 0.5664 - accuracy: 0.7964
Epoch 2/20
7200/7200 [==============================] - 1s 107us/sample - loss: 0.4389 - accuracy: 0.7971
Epoch 3/20
7200/7200 [==============================] - 1s 109us/sample - loss: 0.4311 - accuracy: 0.7971
Epoch 4/20
7200/7200 [==============================] - 1s 112us/sample - loss: 0.4276 - accuracy: 0.7971
Epoch 5/20
7200/7200 [==============================] - 1s 126us/sample - loss: 0.4245 - accuracy: 0.7971
Epoch 6/20
7200/7200 [==============================] - 1s 124us/sample - loss: 0.4222 - accuracy: 0.7971
Epoch 7/20
7200/7200 [==============================] - 1s 128us/sample - loss: 0.4205 - accuracy: 0.7981
Epoch 8/20
7200/7200 [==============================] - 1s 129us/sample - loss: 0.4183 - accuracy: 0.8174
Epoch 9/20
7200/7200 [==============================] - 1s 115us/sample - loss: 0.4165 - accuracy: 0.8229
Epoch 10/20
7200/7200 [=

7200/7200 [==============================] - 1s 104us/sample - loss: 0.4121 - accuracy: 0.8311
Epoch 13/20
7200/7200 [==============================] - 1s 97us/sample - loss: 0.4107 - accuracy: 0.8300
Epoch 14/20
7200/7200 [==============================] - 1s 102us/sample - loss: 0.4102 - accuracy: 0.8333
Epoch 15/20
7200/7200 [==============================] - 1s 128us/sample - loss: 0.4095 - accuracy: 0.8329
Epoch 16/20
7200/7200 [==============================] - 1s 90us/sample - loss: 0.4086 - accuracy: 0.8349
Epoch 17/20
7200/7200 [==============================] - 1s 94us/sample - loss: 0.4081 - accuracy: 0.8353
Epoch 18/20
7200/7200 [==============================] - 1s 89us/sample - loss: 0.4074 - accuracy: 0.8344
Epoch 19/20
7200/7200 [==============================] - 1s 90us/sample - loss: 0.4066 - accuracy: 0.8353
Epoch 20/20
7200/7200 [==============================] - 1s 82us/sample - loss: 0.4062 - accuracy: 0.8368
Train on 7200 samples
Epoch 1/20
7200/7200 [===========

Epoch 4/20
7200/7200 [==============================] - 1s 102us/sample - loss: 0.4238 - accuracy: 0.7944
Epoch 5/20
7200/7200 [==============================] - 1s 107us/sample - loss: 0.4180 - accuracy: 0.7944
Epoch 6/20
7200/7200 [==============================] - 1s 102us/sample - loss: 0.4119 - accuracy: 0.8126
Epoch 7/20
7200/7200 [==============================] - 1s 103us/sample - loss: 0.4049 - accuracy: 0.8249
Epoch 8/20
7200/7200 [==============================] - 1s 102us/sample - loss: 0.3982 - accuracy: 0.8267
Epoch 9/20
7200/7200 [==============================] - 1s 103us/sample - loss: 0.3923 - accuracy: 0.8286
Epoch 10/20
7200/7200 [==============================] - 1s 104us/sample - loss: 0.3874 - accuracy: 0.8293
Epoch 11/20
7200/7200 [==============================] - 1s 103us/sample - loss: 0.3834 - accuracy: 0.8301
Epoch 12/20
7200/7200 [==============================] - 1s 104us/sample - loss: 0.3804 - accuracy: 0.8301
Epoch 13/20
7200/7200 [====================

Epoch 56/100
7200/7200 [==============================] - 1s 116us/sample - loss: 0.4006 - accuracy: 0.8350
Epoch 57/100
7200/7200 [==============================] - 1s 107us/sample - loss: 0.4008 - accuracy: 0.8338
Epoch 58/100
7200/7200 [==============================] - 1s 102us/sample - loss: 0.4002 - accuracy: 0.8354
Epoch 59/100
7200/7200 [==============================] - 1s 109us/sample - loss: 0.4004 - accuracy: 0.8363
Epoch 60/100
7200/7200 [==============================] - 1s 103us/sample - loss: 0.4004 - accuracy: 0.8342
Epoch 61/100
7200/7200 [==============================] - 1s 105us/sample - loss: 0.4000 - accuracy: 0.8350
Epoch 62/100
7200/7200 [==============================] - 1s 106us/sample - loss: 0.4004 - accuracy: 0.8350
Epoch 63/100
7200/7200 [==============================] - 1s 99us/sample - loss: 0.4002 - accuracy: 0.8360
Epoch 64/100
7200/7200 [==============================] - 1s 105us/sample - loss: 0.4006 - accuracy: 0.8357
Epoch 65/100
7200/7200 [=====

7200/7200 [==============================] - 1s 110us/sample - loss: 0.4282 - accuracy: 0.7956
Epoch 6/100
7200/7200 [==============================] - 1s 113us/sample - loss: 0.4267 - accuracy: 0.7956 - loss:
Epoch 7/100
7200/7200 [==============================] - 1s 106us/sample - loss: 0.4236 - accuracy: 0.7956
Epoch 8/100
7200/7200 [==============================] - 1s 111us/sample - loss: 0.4202 - accuracy: 0.8101
Epoch 9/100
7200/7200 [==============================] - 1s 106us/sample - loss: 0.4184 - accuracy: 0.8221 - los
Epoch 10/100
7200/7200 [==============================] - 1s 103us/sample - loss: 0.4167 - accuracy: 0.8240
Epoch 11/100
7200/7200 [==============================] - 1s 103us/sample - loss: 0.4155 - accuracy: 0.8271
Epoch 12/100
7200/7200 [==============================] - 1s 105us/sample - loss: 0.4146 - accuracy: 0.8297
Epoch 13/100
7200/7200 [==============================] - 1s 105us/sample - loss: 0.4132 - accuracy: 0.8306
Epoch 14/100
7200/7200 [=======

...

7200/7200 [==============================] - 1s 74us/sample - loss: 0.4056 - accuracy: 0.8336
Epoch 51/100
7200/7200 [==============================] - 1s 73us/sample - loss: 0.4051 - accuracy: 0.8338
Epoch 52/100
7200/7200 [==============================] - 1s 74us/sample - loss: 0.4053 - accuracy: 0.8335
Epoch 53/100
7200/7200 [==============================] - 1s 71us/sample - loss: 0.4043 - accuracy: 0.8329
Epoch 54/100
7200/7200 [==============================] - 1s 75us/sample - loss: 0.4053 - accuracy: 0.8336
Epoch 55/100
7200/7200 [==============================] - 1s 76us/sample - loss: 0.4043 - accuracy: 0.8332
Epoch 56/100
7200/7200 [==============================] - 1s 74us/sample - loss: 0.4049 - accuracy: 0.8339
Epoch 57/100
7200/7200 [==============================] - 1s 72us/sample - loss: 0.4048 - accuracy: 0.8319
Epoch 58/100
7200/7200 [==============================] - 1s 73us/sample - loss: 0.4043 - accuracy: 0.8318
Epoch 59/100
7200/7200 [==========================

7200/7200 [==============================] - 1s 74us/sample - loss: 0.4600 - accuracy: 0.7969
Epoch 3/100
7200/7200 [==============================] - 0s 68us/sample - loss: 0.4370 - accuracy: 0.7969
Epoch 4/100
7200/7200 [==============================] - 1s 72us/sample - loss: 0.4309 - accuracy: 0.7969
Epoch 5/100
7200/7200 [==============================] - 0s 67us/sample - loss: 0.4275 - accuracy: 0.7969
Epoch 6/100
7200/7200 [==============================] - 1s 70us/sample - loss: 0.4248 - accuracy: 0.7969
Epoch 7/100
7200/7200 [==============================] - 1s 70us/sample - loss: 0.4222 - accuracy: 0.7997
Epoch 8/100
7200/7200 [==============================] - 1s 73us/sample - loss: 0.4198 - accuracy: 0.8194
Epoch 9/100
7200/7200 [==============================] - 1s 71us/sample - loss: 0.4176 - accuracy: 0.8249
Epoch 10/100
7200/7200 [==============================] - 1s 70us/sample - loss: 0.4155 - accuracy: 0.8250
Epoch 11/100
7200/7200 [==============================] -

Epoch 78/100
7200/7200 [==============================] - 1s 73us/sample - loss: 0.3945 - accuracy: 0.8389
Epoch 79/100
7200/7200 [==============================] - 1s 70us/sample - loss: 0.3945 - accuracy: 0.8390
Epoch 80/100
7200/7200 [==============================] - 0s 64us/sample - loss: 0.3941 - accuracy: 0.8378
Epoch 81/100
7200/7200 [==============================] - 1s 73us/sample - loss: 0.3942 - accuracy: 0.8385
Epoch 82/100
7200/7200 [==============================] - 1s 71us/sample - loss: 0.3941 - accuracy: 0.8388
Epoch 83/100
7200/7200 [==============================] - 1s 71us/sample - loss: 0.3945 - accuracy: 0.8390
Epoch 84/100
7200/7200 [==============================] - 1s 70us/sample - loss: 0.3941 - accuracy: 0.8379
Epoch 85/100
7200/7200 [==============================] - 0s 69us/sample - loss: 0.3941 - accuracy: 0.8397
Epoch 86/100
7200/7200 [==============================] - 1s 71us/sample - loss: 0.3939 - accuracy: 0.8400
Epoch 87/100
7200/7200 [=============

7200/7200 [==============================] - 1s 73us/sample - loss: 0.3583 - accuracy: 0.8532
Epoch 31/100
7200/7200 [==============================] - 1s 70us/sample - loss: 0.3570 - accuracy: 0.8558
Epoch 32/100
7200/7200 [==============================] - 1s 72us/sample - loss: 0.3573 - accuracy: 0.8537
Epoch 33/100
7200/7200 [==============================] - 1s 73us/sample - loss: 0.3564 - accuracy: 0.8532
Epoch 34/100
7200/7200 [==============================] - 1s 71us/sample - loss: 0.3558 - accuracy: 0.8537
Epoch 35/100
7200/7200 [==============================] - 1s 73us/sample - loss: 0.3551 - accuracy: 0.8542
Epoch 36/100
7200/7200 [==============================] - 1s 73us/sample - loss: 0.3547 - accuracy: 0.8539
Epoch 37/100
7200/7200 [==============================] - 1s 71us/sample - loss: 0.3536 - accuracy: 0.8564
Epoch 38/100
7200/7200 [==============================] - 1s 77us/sample - loss: 0.3538 - accuracy: 0.8547
Epoch 39/100
7200/7200 [==========================

7200/7200 [==============================] - 0s 64us/sample - loss: 0.3352 - accuracy: 0.8631
Epoch 83/100
7200/7200 [==============================] - 1s 73us/sample - loss: 0.3350 - accuracy: 0.8631
Epoch 84/100
7200/7200 [==============================] - 0s 69us/sample - loss: 0.3350 - accuracy: 0.8636
Epoch 85/100
7200/7200 [==============================] - 1s 72us/sample - loss: 0.3348 - accuracy: 0.8649
Epoch 86/100
7200/7200 [==============================] - 0s 63us/sample - loss: 0.3348 - accuracy: 0.8635
Epoch 87/100
7200/7200 [==============================] - 1s 73us/sample - loss: 0.3347 - accuracy: 0.8626
Epoch 88/100
7200/7200 [==============================] - 1s 71us/sample - loss: 0.3348 - accuracy: 0.8640
Epoch 89/100
7200/7200 [==============================] - 1s 77us/sample - loss: 0.3345 - accuracy: 0.8624
Epoch 90/100
7200/7200 [==============================] - 1s 71us/sample - loss: 0.3351 - accuracy: 0.8614
Epoch 91/100
7200/7200 [==========================

In [22]:
print(best_parameters)
print(best_accuracy)

{'batch_size': 25, 'epochs': 100, 'optimizer': 'adam'}
0.842
